In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.3 MB/s 
     |████████████████████████████████| 182 kB 68.2 MB/s 
     |████████████████████████████████| 7.6 MB 49.0 MB/s 


In [3]:
from transformers import AutoTokenizer, AlbertTokenizer
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer

df = pd.read_parquet("/content/drive/MyDrive/CAU-50648-big-data/kaggle-pog-series-s01e01/dataset.parquet")
train, test = train_test_split(df, test_size=0.2)

In [4]:
train.columns

Index(['index', 'video_id', 'title', 'publishedAt', 'channelId', 'category',
       'channelTitle', 'categoryId', 'trending_date', 'tags', 'view_count',
       'likes', 'dislikes', 'comment_count', 'thumbnail_link',
       'comments_disabled', 'ratings_disabled', 'description', 'id',
       'duration_seconds', 'has_thumbnail', 'target', 'publish_day',
       'publish_hour', 'trending_date-publishedAt', 'tags_len',
       'description_len', 'description_title'],
      dtype='object')

In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.8 MB/s 


In [6]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

train_encodings = tokenizer(list(train["description_title"].values), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test["description_title"].values), truncation=True, padding=True, max_length=128)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [7]:
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = NSMCDataset(train_encodings, train["view_count"].values.astype(np.float32))
test_dataset = NSMCDataset(test_encodings, np.expand_dims(test["view_count"].values.astype(np.float32), 0))

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2.0,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2000
)

In [9]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", problem_type="regression")


Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'pre_class

In [10]:
model.config.dim

768

In [11]:
model.num_labels = 1

In [12]:
model.pre_classifier = nn.Linear(model.config.dim, model.config.dim)
model.classifier = nn.Linear(model.config.dim, 1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 70548
  Num Epochs = 2
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 552
  Number of trainable parameters = 134740241


Step,Training Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=552, training_loss=76.4562740602355, metrics={'train_runtime': 808.1797, 'train_samples_per_second': 174.585, 'train_steps_per_second': 0.683, 'total_flos': 2772696854089008.0, 'train_loss': 76.4562740602355, 'epoch': 2.0})

In [13]:
!nvidia-smi

Sat Nov 26 19:30:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    44W /  70W |  10420MiB / 15109MiB |     79%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------